In [20]:
import numpy as np
import torch
from datetime import datetime, timedelta
from torch.nn import Linear, ModuleList, Parameter
from dyrep_node_Hawkes import DyRepNode

In [22]:
rnd = np.random.RandomState(42)
model1= DyRepNode(num_nodes= 4, 
                  hidden_dim= 2, 
                  random_state= rnd, 
                  first_date=1, 
                  end_datetime=1, 
                  num_neg_samples= 2, 
                  num_time_samples = 5,
                 device='cpu', all_comms=False, train_td_max=None)


In [23]:
for name, param in model1.named_parameters():
    print(name,param.shape,param.data)

w_t torch.Size([1]) tensor([0.5000])
alpha torch.Size([1]) tensor([0.5000])
psi torch.Size([1]) tensor([0.5000])
omega.weight torch.Size([1, 2]) tensor([[ 0.5206, -0.5538]])
omega.bias torch.Size([1]) tensor([0.5191])
W_h.weight torch.Size([2, 2]) tensor([[-0.1358,  0.6934],
        [-0.6190, -0.6382]])
W_h.bias torch.Size([2]) tensor([-0.3202,  0.1341])
W_event_to_neigh.weight torch.Size([2, 2]) tensor([[-0.4343, -0.3727],
        [-0.1324, -0.3978]])
W_event_to_neigh.bias torch.Size([2]) tensor([-0.5827, -0.4955])
W_rec_event.weight torch.Size([2, 2]) tensor([[ 0.1638, -0.6388],
        [ 0.2997,  0.5835]])
W_rec_event.bias torch.Size([2]) tensor([-0.2839, -0.2384])
W_rec_neigh.weight torch.Size([2, 2]) tensor([[-0.5674, -0.5742],
        [ 0.4669,  0.1483]])
W_rec_neigh.bias torch.Size([2]) tensor([ 0.6470, -0.3416])
W_t.weight torch.Size([2, 4]) tensor([[-0.2926, -0.0436, -0.0085, -0.3629],
        [-0.4958,  0.3651,  0.3907,  0.2285]])
W_t.bias torch.Size([2]) tensor([-0.2686, -0.

In [24]:
prev_embedding = [[1,1],[2,2],[3,3],[4,4]]
prev_embedding=torch.Tensor(prev_embedding)
time_delta_it =torch.Tensor([[2,3,1],[2,3,1],[2,3,1],[2,3,1]])
u_neighborhood = [1,2]
u_event = 0

In [16]:
model.update_node_embedding_without_attention(prev_embedding, u_event=u_event, u_neighborhood=u_neighborhood, time_delta_it=time_delta_it)

RuntimeError: shape '[2, 4]' is invalid for input of size 6